<a href="https://colab.research.google.com/github/cgmiller/Genome-on-the-Go/blob/main/Genome_on_the_Go_Workshop_12Feb2026.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🧬 Nanopore Sequencing Analysis: Chloroplast Genome
**Student Workshop Notebook**

## Overview
In this notebook, we will process raw sequencing data from an **Oxford Nanopore MinION** sequencer.

## Prerequisites
You must have the following two files ready to upload:
1.  **Sequencing Data:** A `.fastq` file containing your raw reads.
2.  **Reference Genome:** A `.fasta` file of the chloroplast genome you expect to find (e.g., *Arabidopsis thaliana* chloroplast).

## Workflow
1.  **Setup:** Install bioinformatics tools.
2.  **Upload:** Load your real experimental data.
3.  **QC & Stats:** Assess the quality of your sequencing run.
4.  **Filtering:** Remove short or low-quality reads.
5.  **Mapping:** Align reads to your reference genome.
6.  **Consensus / Genome Assembly:** Generate the final biological sequence.

---


## Part 1: Environment Setup


In [ ]:
# @title Step 1: Install Libraries and Tools
# Installing Biopython (Python library) and Minimap2/Samtools/Bcftools (Bioinformatics software)
!pip install biopython pandas seaborn matplotlib
!apt-get install minimap2 samtools bcftools

import sys
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from Bio import SeqIO
from google.colab import files

print("✅ Environment ready!")


## Part 2: Upload Your Data

**Instructions:**
1. Run the cell below.
2. Click the **'Choose Files'** button.
3. Upload your `.fastq` file AND your `.fasta` reference file.
4. The code will automatically rename them to standard names so the rest of the notebook works smoothly.


In [ ]:
# @title Step 2: Upload Data
uploaded = files.upload()

# DEFINE STANDARD FILENAMES
reads_filename = "chloroplast_reads.fastq"
ref_filename = "chloroplast_reference.fasta"

# LOGIC TO RENAME UPLOADED FILES
print("\nProcessing uploads...")
found_reads = False
found_ref = False

for filename in uploaded.keys():
    if filename.endswith(".fastq") or filename.endswith(".fq"):
        os.rename(filename, reads_filename)
        print(f"✅ Reads identified: '{filename}' -> renamed to '{reads_filename}'")
        found_reads = True
    elif filename.endswith(".fasta") or filename.endswith(".fa") or filename.endswith(".fna"):
        os.rename(filename, ref_filename)
        print(f"✅ Reference identified: '{filename}' -> renamed to '{ref_filename}'")
        found_ref = True

if not found_reads:
    print("❌ ERROR: No .fastq file found! Please re-run and upload your sequencing data.")
if not found_ref:
    print("❌ ERROR: No .fasta reference file found! Please re-run and upload your reference genome.")

if found_reads and found_ref:
    print("\n🎉 Data ready for analysis!")


## Part 3: Basic Statistics (The "Table 1" of Genomics)


In [ ]:
# @title Step 3: Parse and Calculate Basic Stats
def calculate_fastq_stats(fastq_file):
    lengths = []
    qualities = []
    print(f"Parsing {fastq_file}...")

    # Iterate through every read in the file
    for record in SeqIO.parse(fastq_file, "fastq"):
        lengths.append(len(record))
        avg_q = np.mean(record.letter_annotations["phred_quality"])
        qualities.append(avg_q)

    lengths = np.array(lengths)
    qualities = np.array(qualities)

    if len(lengths) == 0:
        print("⚠️ Warning: File is empty or not in FASTQ format.")
        return {}, [], []

    # Calculate N50 (Weighted Median)
    sorted_lengths = np.sort(lengths)
    cumsum = np.cumsum(sorted_lengths)
    n50_idx = np.searchsorted(cumsum, cumsum[-1] / 2)
    n50 = sorted_lengths[n50_idx]

    stats = {
        "Total Reads": len(lengths),
        "Total Yield (bp)": np.sum(lengths),
        "Mean Length (bp)": int(np.mean(lengths)),
        "Max Length (bp)": np.max(lengths),
        "N50 (bp)": int(n50),
        "Mean Q-Score": np.round(np.mean(qualities), 2)
    }
    return stats, lengths, qualities

# Run analysis on the renamed file
raw_stats, raw_lengths, raw_quals = calculate_fastq_stats(reads_filename)

print("\n--- Raw Data Statistics ---")
display(pd.DataFrame(raw_stats, index=["Value"]).T)


## Part 4: Visualization


In [ ]:
# @title Step 4: Visualize Read Length vs. Quality
if len(raw_lengths) > 0:
    df = pd.DataFrame({
        "Read Length (bp)": raw_lengths,
        "Mean Quality (Phred)": raw_quals
    })

    # Plot 1: Length vs Quality Hexbin
    plt.figure(figsize=(10, 6))
    sns.jointplot(x="Read Length (bp)", y="Mean Quality (Phred)", data=df, kind="hex", color="#4CB391")
    plt.suptitle("Read Length vs. Quality Score", y=1.02)
    plt.show()

    # Plot 2: Length Distribution
    plt.figure(figsize=(10, 5))
    sns.histplot(raw_lengths, log_scale=True, color="skyblue")
    plt.title("Read Length Distribution (Log Scale)")
    plt.xlabel("Read Length (bp)")
    plt.show()
else:
    print("No data to plot.")


## Part 5: Quality Filtering
Real data often contains "junk" reads (too short or low quality). We will filter them out.


In [ ]:
# @title Step 5: Filter the Data
# FILTER SETTINGS
min_length = 1000  # Minimum length in base pairs
min_quality = 10   # Minimum average Phred quality score (Q10 = 90% accuracy)

filtered_output = "filtered_reads.fastq"
filtered_records = []

print(f"Filtering reads (Min Length: {min_length}, Min Q: {min_quality})...")

for record in SeqIO.parse(reads_filename, "fastq"):
    avg_q = np.mean(record.letter_annotations["phred_quality"])
    if len(record) >= min_length and avg_q >= min_quality:
        filtered_records.append(record)

# Save filtered file
with open(filtered_output, "w") as output_handle:
    SeqIO.write(filtered_records, output_handle, "fastq")

print(f"✅ Filtering complete. Saved to: {filtered_output}")
print(f"Reads kept: {len(filtered_records)} / {raw_stats['Total Reads']}")
print(f"Reads discarded: {raw_stats['Total Reads'] - len(filtered_records)}")


## Part 6: Verify Improvements


In [ ]:
# @title Step 6: Comparative Analysis
filt_stats, _, _ = calculate_fastq_stats(filtered_output)

comparison = pd.DataFrame({
    "Raw Data": raw_stats,
    "Filtered Data": filt_stats
})
print("\n--- Improvement Analysis ---")
display(comparison)


## Part 7: Map Reads to Reference
We will use `minimap2` to align our filtered reads to the reference genome you uploaded.


In [ ]:
# @title Step 7: Map Reads (Minimap2)
# Check if reference exists
if not os.path.exists(ref_filename):
    print("❌ ERROR: Reference file missing. Please go back to Step 2.")
else:
    print(f"Mapping {filtered_output} to {ref_filename}...")

    # 1. Align reads (-ax map-ont is optimized for Nanopore)
    !minimap2 -ax map-ont {ref_filename} {filtered_output} > aligned.sam

    # 2. Convert to BAM, Sort, and Index
    print("Processing alignment files...")
    !samtools view -S -b aligned.sam > aligned.bam
    !samtools sort aligned.bam -o aligned.sorted.bam
    !samtools index aligned.sorted.bam

    print("✅ Mapping complete! Output: aligned.sorted.bam")


## Part 8: Visualize Coverage Depth
Let's see how many reads covered each part of the chloroplast genome.


In [ ]:
# @title Step 8: Plot Genome Coverage
if os.path.exists("aligned.sorted.bam"):
    !samtools depth aligned.sorted.bam > coverage.txt

    # Check if coverage file is empty (mapping failed?)
    if os.stat("coverage.txt").st_size == 0:
        print("⚠️ Warning: Coverage file is empty. Your reads might not match the reference genome!")
    else:
        cov_df = pd.read_csv("coverage.txt", sep="\t", names=["Chrom", "Position", "Depth"])
        mean_depth = cov_df["Depth"].mean()

        plt.figure(figsize=(12, 6))
        plt.fill_between(cov_df["Position"], cov_df["Depth"], color="#2c3e50", alpha=0.3)
        plt.plot(cov_df["Position"], cov_df["Depth"], color="#2c3e50", linewidth=1)
        plt.axhline(mean_depth, color='red', linestyle='--', label=f"Mean: {mean_depth:.0f}x")

        plt.title("Genome Coverage Map")
        plt.xlabel("Genome Position (bp)")
        plt.ylabel("Read Depth (x)")
        plt.legend()
        plt.show()
else:
    print("❌ BAM file missing. Run Step 7 first.")


## Part 9: Consensus Genome Assembly
Create the final consensus sequence based on the mapping.


In [ ]:
# @title Step 9: Call Variants & Assembly
print("1. Calling Variants...")
!bcftools mpileup -f {ref_filename} aligned.sorted.bam | bcftools call -mv -Oz -o calls.vcf.gz

print("2. Indexing...")
!bcftools index calls.vcf.gz

print("3. Creating Consensus...")
!bcftools consensus -f {ref_filename} calls.vcf.gz > final_consensus.fasta

print("\n✅ ASSEMBLY COMPLETE: final_consensus.fasta")


## Part 10: Download Results


In [ ]:
# @title Step 10: Zip and Download
print("Zipping files...")
!zip -r analysis_results.zip final_consensus.fasta coverage.txt aligned.sorted.bam calls.vcf.gz

from google.colab import files
files.download("analysis_results.zip")
